# ASSIGNMENT 3 - NIKO PATERNITI BARBINO 638257

**Implement from scratch an RBM and apply it to DSET3. The RBM should be implemented fully by you (both CD-1 training and inference steps) but you are free to use library functions for the rest (e.g. image loading and management, etc.).**

1.     Train an RBM with a number of hidden neurons selected by you (single layer) on the MNIST data (use the training set split provided by the website).

2.     Use the trained RBM to encode a selection of test images (e.g. using one per digit type) using the corresponding activation of the hidden neurons.

3.     Reconstruct the original test images from their hidden encoding and confront the reconstructions with the original image (use a suitable quantitative metric to assess the reconstraction quality and also choose few examples to confront visually).

# Utility functions

In [47]:
def logistic(x):  
    return 1.0 / (1 + np.exp(-x))

def flatten(x):
    return x.reshape(x.shape[0], -1)

# RBM IMPLEMENTATION


We implemented the RBM randomly initializing the weights and initializing the bias to zero for each hidden and visible unit.
Follows the implementation of the CD-1 Trainining algorithm

In [48]:
import numpy as np

class RBM:  
    def __init__(self, hidden_units, visible_units):  
        self.nh = hidden_units  
        self.nv = visible_units
        self.weights = np.random.uniform(-1/self.nv, 1/self.nv, (self.nv, self.nh))  
        self.bias_h = np.zeros(self.nh)  
        self.bias_v = np.zeros(self.nv) 
        print("Built a RBM with " + str(self.nv) + " visible units and " + str(self.nh) + " hidden units")

    def train(self, Xtr, epochs = 100, learning_rate = 0.1):  # CD-1 training algorithm
        n = 6000  # batch size

        print("Training on " + str(n) + " random elements for " + str(epochs) + " epochs")
        for epoch in range(epochs):
            idx = np.random.uniform(low = 0, high = Xtr.shape[0], size=n).astype(int)
            cXtr = Xtr[idx,:]

            # Hidden probability
            h_prob = logistic(np.dot(cXtr, self.weights) + self.bias_h)
            wake = np.dot(cXtr.T, h_prob)

            # Hidden states
            h_state = h_prob > np.random.rand(n, self.nh)

            # Reconstruction probability
            reconstruction_data_prob = logistic(np.dot(h_state, self.weights.T) + self.bias_v)

            # Reconstructed data
            reconstruction_data = reconstruction_data_prob > np.random.rand(n, self.nv)
            h_neg_prob = logistic(np.dot(reconstruction_data, self.weights) + self.bias_h)
            dream = np.dot(reconstruction_data.T, h_neg_prob)

            # Learning phase
            error = np.sum((cXtr - reconstruction_data)**2)/n
            dW = (wake - dream)/n
            dBh = (np.sum(h_prob) - np.sum(h_neg_prob))/n
            dBv = (np.sum(cXtr) - np.sum(reconstruction_data))/n
            self.weights += learning_rate*dW
            self.bias_h += learning_rate*dBh
            self.bias_v += learning_rate*dBv
            print("\rError:\t" + "{:.5f}".format(error), end="")
        print("")


    def get_hidden_activations(self, Xtr): 
        n = Xtr.shape[0]

        print("Computing hidden activations for " + str(n) + " elements")
        h_states = np.ones((n, self.nh))

        h_prob = logistic(np.dot(Xtr, self.weights) + self.bias_h)
        h_states[:,:] = h_prob > np.random.rand(n, self.nh)
        return h_states

We load the dataset from keras and compute the hidden activations

In [49]:
from keras.datasets import mnist

(X_train, y_train), (X_test, y_test) = mnist.load_data()
print(str(X_train.shape[0]) + " images of " + str(X_train.shape[1]) + "x" + str(X_train.shape[2]) + " pixels =>", end = " ")
Xtr = flatten(X_train)
Xts = flatten(X_test)
print(str(Xtr.shape[0]) + " vectors " + str(Xtr.shape[1]) + " elements")

rbm = RBM(visible_units = (28*28), hidden_units = 500)
rbm.train(Xtr, epochs=100, learning_rate=0.01) 
X_train_h = rbm.get_hidden_activations(Xtr)
X_test_h = rbm.get_hidden_activations(Xts)    

60000 images of 28x28 pixels => 60000 vectors 784 elements
Built a RBM with 784 visible units and 500 hidden units
Training on 6000 random elements for 100 epochs
Error:	11348.19600
Computing hidden activations for 60000 elements
Computing hidden activations for 10000 elements


# Reconstruct the original test images from their hidden encoding and confront the reconstructions with the original image (use a suitable quantitative metric to assess the reconstraction quality and also choose few examples to confront visually).